# Customizing the Parameter File for Binary Neutron Stars Runs with the Einstein Toolkit 
---
#### by Maria C Hamilton

## Parametrized Piecewise Polytropic Equations of State
---

### Preliminaries
---
* Binary Neutron Stars Initial Data generated with the code [LORENE](https://lorene.obspm.fr/) 
- An excellent collection of initial data can be found [here:](https://ccrgpages.rit.edu/~jfaber/BNSID/Data/)
* Access to a cluster with the Einstein Toolkit running on it.

### Background
---
* A polytropic EOS is described by: $P=K \rho^{\Gamma}$ with $K$ polytropic constant and $\Gamma$ polytropic index
* Read about parametrized polytropic equations of state (*EOS*) presented in 
    - https://arxiv.org/pdf/0812.2163.pdf 

The authors build a model for various 7-pieces hybrid piecewise polytropes EOS, where the first 4 pieces, below nuclear density, are fixed to the nucleonic Skyrme-Lyon equation of state, and the next 3 parameters are freely specified by the user.

### Path and Type of  EOS 
---
We start with the parameter file from the Einstein Toolkit Gallery **nsnstohmns.par**.
- First, we change the path to upload the desired initial data for the parametrized EOS: 

``Meudon_Bin_NS::filename = "<path to resu.d>" ``

- Next, we must add a parameter for the EOS type to **Meudon_Bin_NS**: 

`` Meudon_Bin_NS::eos_table = "Hybrid" ``

- Lastly, we must change the type of the EOS for **GRHydro**, as well: 

`` GRHydro::GRHydro_eos_table = "Ideal_Fluid" `` 

becomes 

`` GRHydro::GRHydro_eos_table = "Hybrid" ``

### Athmosphere and Number of Pieces 
---
* We must keep the previous values for $k$ and $\Gamma$ for the one-piece polytropic EOS, already included in **nsnstohmns.par**, because they are necessary for the hybrid EOS as the athmosphere:

``EOS_Omni::poly_K = 123.64110496340211``

``EOS_Omni::poly_gamma = 2.0`` *# already specified in  EOS_Omni*

* We specify the number of the hybrid pieces, and the thermal component $\Gamma_{th}$ corresponding to a cold EOS (see https://arxiv.org/abs/1006.3315):

`` EOS_Omni::n_pieces = 7``

``EOS_Omni::hybrid_gamma_th = 1.8``

Next, the thorn **EOS_Omni** requires the parameters ``hybrid_k0``, ``hybrid_gamma[0..6]`` and ``hybrid_rho[0...6]`` necessary for the specific EOS. 
Out of those parameters, all except ``hybrid_gamma[4]`` are known and can be taken directly, either from https://arxiv.org/pdf/0812.2163.pdf or https://arxiv.org/pdf/2008.03342.pdf. 

### Crust and the Fixed Part of the Parametrized Piecewise EOS
---
* The first 4 pieces of the 7 pieces hybrid EOS correspond to a 4 pieces piecewise polytropic nucleonic Skyrme-Lyon (*SLy(4)*) equation of state.

* These values are taken directly from https://arxiv.org/pdf/0812.2163.pdf

* Below is Table II, from the 2008 paper by *Read et al* https://arxiv.org/pdf/0812.2163.pdf, giving the necessary quantities for the *Sly* piecewise polytrope (*PP*) model in *cgs* units. 

| $K_i$ | $\Gamma_i$ | $\rho_i$| 
| --- | --- | --- | 
| 6.80110 e-09 | 1.58425 | 2.44034 e+07 | 
| 1.06186 e-06 | 1.28733 | 3.78358 e+11 | 
| 5.32697 e+01 | 0.62223 | 2.62780 e+12 | 
| 3.99874 e-08 | 1.35692 | - |

#### Fixed *Sly* EOS for $\Gamma_i$
---
* As explained above, all the coefficients ``hybrid_gammas`` are known and can be entered directly because they are unitless; for example, for the piecewise polytrope (PP) model they will be:

``EOS_Omni::hybrid_gamma[0] = 1.58425 ``

``EOS_Omni::hybrid_gamma[1] = 1.28733 ``

``EOS_Omni::hybrid_gamma[2] = 0.62223 ``

``EOS_Omni::hybrid_gamma[3] = 1.35692 ``

#### Fixed *Sly* EOS for $\rho_i$
---
* We will add next the three densities to the parameter file, but before that, we must transform
    - from *cgs* units $g/cm^3$ to *SI* units $kg/m^3$
    - from *mks* units to Cactus Units *CU* 
    
##### Cactus Units
* The code uses a geometrized system of units, in which the universal constants become unitless and unity: $G=c=1$, the mass is expressed in terms of the solar mass $M_{\cdot}$, which then is geometrized to 1 to represent **Cactus units** (*CU*). All the other *mks* units are expressed in terms of the mass. The transormation from Cactus units to *mks* units goes as follows:  

In [1]:
# units transformation from mks to Cactus units
M0 = 1.98892*pow(10,30)
G = 6.67428*pow(10,-11)
c = 2.9979*pow(10,8)
M_mks = M0 # mass
L_mks = M_mks*G*pow(c,-2) #length
T_mks = M_mks*G*pow(c,-3) #time
rho_mks_CU = M_mks/pow(L_mks,3)
print(rho_mks_CU)

6.172412699782906e+20


* To transform the density from *mks* to Cactus units we do the following:
    
    $$\rho[{CU}] = \frac{\rho[{mks}]}{\rho_{mks\rightarrow CU}}$$
where $\rho_{mks\rightarrow CU} = 6.1727\times 10^{20}$ is the constant that transforms the density from *mks* to Cactus units.
*For a good explanation check  https://en.wikipedia.org/wiki/Geometrized_unit_system and
https://arxiv.org/pdf/physics/0110060.pdf.*

##### Sample Calculation
* Let's take the density from the *PP* model of *SLy* EOS table above in *cgs* and transform in Cactus units: 
    $$\rho_{0} = 2.44034 \times 10^{7}~[cgs] 
    = 2.44034 \times 10^{10}~[mks] \\
    = 3.95384\times 10^{-11}~[CU]$$ 

In [2]:
import numpy as np
rho_SLy4 = np.array([2.44034*pow(10,7),3.78358*pow(10,11),2.62780*pow(10,12)])
rho_SLy4_CU = rho_SLy4*pow(10,3)/rho_mks_CU
print(rho_SLy4_CU)

[3.95362416e-11 6.12982343e-07 4.25733036e-06]


* Now we can add to the parameter file the first three fixed densities that describe the $Sly$ hybrid EOS chosen.

``EOS_Omni::hybrid_rho[0] = 3.951156e-11`` 
        
``EOS_Omni::hybrid_rho[1] = 6.125960e-07``
        
``EOS_Omni::hybrid_rho[2] = 4.254672e-06``

#### Fixed *Sly* EOS for $K_0$
The last component for the fixed part of the EOS is $K_0$, the polytropic coefficient of the first piece. 
We calculate $K_0$ in Cactus units as the ratio:
$$K_{0}[CU] = \frac{p_{0}[CU]}{\rho_{0}[CU]^{Γ_0}}$$
We already calculated $\rho_0$ in Cactus units, let's calculate now $p_0$ in Cactus units. The unit for pressure has the following value in Cactus units:

In [3]:
F_mks = M_mks*L_mks*pow(T_mks,-2)# force
p_mks = F_mks/pow(L_mks,2)
print(p_mks)

5.547396911836889e+37


There is a subtlety we must account for when calculating the pressure.
[Read et al](https://arxiv.org/pdf/2008.03342.pdf) absorbed the speed of light $c$ into the definition of the pressure, such that it has the same *cgs* unit as the density.
This means that we must rescale with $c^2$ the pressure we compute from the EOS equation to transform it to *mks* units, in the following way:
$pressure = \frac{energy}{volume} = c^2 \frac{mass}{volume}$.
Thus, the equation of state is corrected to *cgs* units this way: 
$$P/c^2 = p = K_i \rho_i^{\Gamma_{i}}$$
where the pressure $P$ is now measured in *dyne/cm^2* and the speed of light is measured in *cm/s*.
When we transform this to *mkg* we get:
$$P[mks] = 0.1 P[cgs]
        = 0.1 c^2[cgs] p
        = 10^3 c^2[mks] p$$

We present the calculation below:

In [4]:
K0 = 6.80110*pow(10,-9)
rho0 = rho_SLy4[0] #2.44034*pow(10,7)
Gamma0 = 1.58425
p0 = K0*pow(rho0,Gamma0)
p0_SI = p0*pow(c,2)*pow(10,3)
p0_CU = p0_SI/p_mks
rho0_CU = rho_SLy4_CU[0]
K0_CU = p0_CU/pow(rho0_CU,Gamma0)
print(K0_CU)

168.52039510283151


Finally, we enter the value calulated in the parameter:

``EOS_Omni::hybrid_k0 = 1.685819e2``

### Specific EOS Parameters
---

* Before we will start to calculate the necessary quantities, we emphasize that in https://arxiv.org/pdf/0812.2163.pdf the central density of each EOS is fixed at $\rho_5 = 10^{15} g/cm^3$ and the next to central density is fixed at $\rho_4 = 10^{14.7} g/cm^3$.

* We transform those densities in Cactus units and enter them in the parameter file: 

``EOS_Omni::hybrid_rho[4] = 8.11931e-04``

``EOS_Omni::hybrid_rho[5] = 1.62002e-03``

In [5]:
rhoC = np.array([pow(10,14.7),pow(10,15)])
rhoC_CU = rhoC*pow(10,3)/rho_mks_CU
print(rhoC_CU)

[0.00081198 0.00162011]


* The three specific polytropic indices for the 7 pieces EOS are also known, and are given in **Table III** from https://arxiv.org/pdf/0812.2163.pdf. As example, below are the specific EOS parameters for the 7 pieces *SLy* EOS: 

| EOS | $\lg P$ | $\Gamma_4$ | $\Gamma_5$ | $\Gamma_6$ |
| --- | --- |--- | --- | --- | 
| SLy |  34.384 | 3.005 | 2.988 | 2.851 |

* Those values for $\Gamma_i$ will be entered as they are, because they are unitless. 

``EOS_Omni::hybrid_gamma[4] = 3.005 ``

``EOS_Omni::hybrid_gamma[5] = 2.988 ``

``EOS_Omni::hybrid_gamma[6] = 2.851 ``

* Thus, the only unknown part, and the most important one, is the matching density $\rho_3$.

### Shortcut to calculating $\rho_3$

* One can take a shortcut, and simply use the values from the densities $\rho_i$ directly from the  file ``par_eos.d`` provided by **LORENE**, which gives the exponent of densities, namely $\lg \rho$.
* For example, in this file, we find that for the *SLy* EOS, $\lg \rho_3 = 14.165$. From here we calculate:  
$$\rho_3 = 1.46218\times 10^{14}~(cgs) = 0.000236875~(CU)$$ 

* We add the final and most important piece to our parameter file:

``EOS_Omni::hybrid_rho[3] = 2.36875e-04``

### The long way to calculating $\rho_3$

* Another way of computing $\rho_3$, if ``par_eos.d`` is not available, is by using the reccurence relation provided in https://arxiv.org/pdf/0812.2163.pdf.

* The condition for the continuity of pressure at a density $\rho_i$ reads: $$p(\rho_i) = K_i \rho_i^{\Gamma_{i}} = K_{i+1} {\rho_i}^{\Gamma_{i+1}}$$
* This leads to the important relation:
$$K_{i+1} = K_i \rho_i^{Γ_i - \Gamma_{i+1}}$$

* We are provided in https://arxiv.org/pdf/0812.2163.pdf with $\lg p_4$ and will use again the *SLy* EOS  as example. 

* Let's first calculate the pressure
$$P_4 = 10^{\lg p} = 2.42103 \times 10^{34} ~(cgs).$$
We must transform the pressure in units of density, in order to plug it into the EOS, so we need to divide by $c^2$, thus $p_4 = P_4/ c^2$
* We obtain that 
$P_4 = 10^4 c^2 K_4 \rho_4^{\Gamma_4}$, where $K_4$ is given by the recurrence relation $K_4  = K_3 \rho_3^{\Gamma_3 - \Gamma_4}$.
* The only unknown is $\rho_3$, and we solve for it. 

In [6]:
logp4 = 34.384
p4 = pow(10, logp4)/(pow(10,4)*pow(c,2))
print("%e" % p4)
rho4 = rhoC[0]
Gamma3 = 1.35692
Gamma4 = 3.005
K3 = 3.99874 * pow(10,-8)

2.693802e+13


In [7]:
from sympy import symbols, Eq, solve
rho3 = symbols('rho3')

K4 = K3*rho3 ** (Gamma3 - Gamma4)
print(K4)

3.99874e-8*rho3**(-1.64808)


In [8]:
eq = p4 - K4 * pow(rho4,Gamma4)
print(eq)

26938022773765.2 - 5.96242826509895e+36*rho3**(-1.64808)


We obtained a relationship between $p_4$ and $\rho_3$:
$$2.69380 \times 10^{13}
= \frac{5.96242 \times 10^{36}}{\rho_3^{1.64808}}$$
* Solving this equation we obtain again: $\rho_3 =  1.46218585922\times 10^{14}.$

* We could also check the recurrence relationship, and calculate the missing polytropic constants:

In [9]:
Ki = np.array([6.80110*pow(10,-9),1.06186*pow(10,-6),5.32697*pow(10,+1),3.99874*pow(10,-8),1.80663*pow(10,-31),3.21196*pow(10,-31),3.64558*pow(10,-29)])
Gammai = np.array([1.58425,1.28733,0.62223,1.35692,3.005,2.988,2.851])
rhoi = np.array([2.44034*pow(10,7),3.78358*pow(10,11),2.62780*pow(10,12),1.4621858592205103*pow(10,14),5.01187*pow(10,14.7),pow(10,15)])
for i in range(6):
    Ki[i+1] = Ki[i]*pow(rhoi[i],(Gammai[i] - Gammai[i+1]))
    print(Ki[i+1])

1.0618444833278537e-06
53.275084583961515
3.9992069172431924e-08
1.8068263838794974e-31
3.3015389117287475e-31
3.747282373175135e-29


* We see that we obtain a vey good agreement with the given $K_{(0...4)}$
* We do not need to go further, but we could fill in the previous **Table 2** by calculating the pressure as well:

In [10]:
pi = np.empty(shape=6)
for i in range(6):
    pi[i] = Ki[i+1]*pow(rhoi[i],Gammai[i+1])*(pow(10,4)*pow(c,2))
    print("%e" %pi[i])

3.088742e+24
7.660850e+29
2.558594e+30
5.975627e+32
3.072912e+36
1.960427e+35


* We do not need to go further, but we could fill in the previous **Table 2** 

| EOS piece | $K_i$ | $\Gamma_i$ | $\rho_i$ |  $p_i$$dyne/cm^2$|
| --- | --- | --- | --- | --- |
| 0 | 6.80110 e-09 | 1.58425 | 2.44034 e+07 | 3.0893 e+24 |
| 1 | 1.06186 e-06 | 1.28733 | 3.78358 e+11 | 7.6609 e+29 |
| 2 | 5.32697 e+01 | 0.62223 | 2.62780 e+12 | 2.55836 e+30 |
| 3 | 3.99874 e-08 | 1.35692 | 1.46219 e+14 |5.97492 e+32|
| 4 | 1.80663 e-31 | 3.005   | 5.01187 e+14 | 2.42103 e+34  |
| 5 | 3.21196 e-31 | 2.988   | 1. e+15 | 1.90722 e+35 |
| 6 | 3.64558 e-29 | 2.851   | -  | -  |

### Special thanks
* Joshua Faber, RIT
* Tanmayee Gupte, RIT 
* Leonardo Werneck, WVU